In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By # Importar By para localizar elementos
import time # Para adicionar pausas, se necessário
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import sys
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [37]:
class UnidadeUSP:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []

    def __str__(self):
        return f"Unidade: {self.nome, self.cursos}"
    
    @staticmethod
    def carregar_unidades(driver, limite=None):
        unidades = []
        combo_unidade = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboUnidade"))
        )
        WebDriverWait(driver, 10).until(lambda d: len(Select(combo_unidade).options) > 1)
        select_unidade = Select(combo_unidade)
        #print(select_unidade.options)
        total_opcoes = len(select_unidade.options)
        if total_opcoes < 2:  # Deve ter pelo menos 1 opção além do vazio
            raise ValueError("Nenhuma unidade carregada no comboUnidade.")
    
        # Limitar ao menor valor entre o número de opções e o limite especificado
        limite = min(limite, total_opcoes - 1)  
       
        # Carregar as unidades
        for i in range(1, 4):  # Ignorar a primeira opção vazia
            nome_unidade = select_unidade.options[i].text.strip()
            if nome_unidade:  # Ignorar opções vazias
                unidade = UnidadeUSP(nome_unidade)

                # Selecionar a unidade e carregar os cursos
                select_unidade.select_by_index(i)
                unidade.cursos = CursoUSP.carregar_cursos(driver)

                unidades.append(unidade)
        return unidades

    def carregar_cursos(self, driver):
        combo_unidade = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboUnidade"))
        )
        select_unidade = Select(combo_unidade)
        
        # Selecionar a unidade
        for option in select_unidade.options:
            if self.nome in option.text:
                select_unidade.select_by_visible_text(option.text)
                break
        
        # Esperar os cursos serem carregados
        WebDriverWait(driver, 10).until(
            lambda d: len(Select(d.find_element(By.ID, "comboCurso")).options) > 1
        )
        
        # Carregar cursos
        self.cursos = CursoUSP.carregar_cursos(driver)
        

In [38]:
class CursoUSP:
    @staticmethod
    def carregar_cursos(driver):
        combo_curso = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboCurso"))
        )
        select_curso = Select(combo_curso)
        
        cursos = []
        for option in select_curso.options:
            nome_curso = option.text.strip()
            if nome_curso:  # Ignorar opções vazias
                curso = CursoUSP(nome=nome_curso)
                curso.carregar_detalhes(driver)
                cursos.append(curso)
        return cursos

    def __init__(self, nome):
        self.nome = nome
        self.duracao_ideal = None
        self.duracao_minima = None
        self.duracao_maxima = None
        self.disciplinas_obrigatorias = []
        self.disciplinas_optativas_livres = []
        self.disciplinas_optativas_eletivas = []

    def carregar_detalhes(self, driver):
        # Localizar e clicar no link do curso para abrir a página de detalhes
        curso_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, self.nome))
        )
        curso_link.click()

        # Esperar a página carregar
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "detalhesCurso"))
        )

        # Extrair informações
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Exemplo: Extraindo duração
        duracao_ideal_elem = soup.find("span", {"id": "duracaoIdeal"})
        if duracao_ideal_elem:
            self.duracao_ideal = duracao_ideal_elem.text.strip()

        duracao_minima_elem = soup.find("span", {"id": "duracaoMinima"})
        if duracao_minima_elem:
            self.duracao_minima = duracao_minima_elem.text.strip()

        duracao_maxima_elem = soup.find("span", {"id": "duracaoMaxima"})
        if duracao_maxima_elem:
            self.duracao_maxima = duracao_maxima_elem.text.strip()

        # Exemplo: Extraindo listas de disciplinas
        obrigatorias = soup.find("ul", {"id": "disciplinasObrigatorias"})
        if obrigatorias:
            self.disciplinas_obrigatorias = [
                li.text.strip() for li in obrigatorias.find_all("li")
            ]

        optativas_livres = soup.find("ul", {"id": "disciplinasOptativasLivres"})
        if optativas_livres:
            self.disciplinas_optativas_livres = [
                li.text.strip() for li in optativas_livres.find_all("li")
            ]

        optativas_eletivas = soup.find("ul", {"id": "disciplinasOptativasEletivas"})
        if optativas_eletivas:
            self.disciplinas_optativas_eletivas = [
                li.text.strip() for li in optativas_eletivas.find_all("li")
            ]

        # Retornar para a página anterior
        driver.back()

In [40]:
def main():
    driver = webdriver.Chrome()
    driver.get("https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275")

    unidades = UnidadeUSP.carregar_unidades(driver, limite=3)
    
    for unidade in unidades:
        print(f"Carregando cursos para a unidade: {unidade.nome}")
        print(unidades.)
        for curso in unidade.cursos:
            print(f"Curso: {curso.nome}")
            print(f"Duração ideal: {curso.duracao_ideal}")
            print(f"Duração mínima: {curso.duracao_minima}")
            print(f"Duração máxima: {curso.duracao_maxima}")
            print(f"Disciplinas obrigatórias: {curso.disciplinas_obrigatorias}")
            print(f"Disciplinas optativas livres: {curso.disciplinas_optativas_livres}")
            print(f"Disciplinas optativas eletivas: {curso.disciplinas_optativas_eletivas}")

    driver.quit()

if __name__ == "__main__":
    main()

Carregando cursos para a unidade: Escola de Artes, Ciências e Humanidades - ( EACH )
[<__main__.UnidadeUSP object at 0x74a5b52f7d40>, <__main__.UnidadeUSP object at 0x74a5b52f7980>, <__main__.UnidadeUSP object at 0x74a5b52f44a0>]
Carregando cursos para a unidade: Escola de Comunicações e Artes - ( ECA )
[<__main__.UnidadeUSP object at 0x74a5b52f7d40>, <__main__.UnidadeUSP object at 0x74a5b52f7980>, <__main__.UnidadeUSP object at 0x74a5b52f44a0>]
Carregando cursos para a unidade: Escola de Educação Física e Esporte - ( EEFE )
[<__main__.UnidadeUSP object at 0x74a5b52f7d40>, <__main__.UnidadeUSP object at 0x74a5b52f7980>, <__main__.UnidadeUSP object at 0x74a5b52f44a0>]


In [53]:

# Funções auxiliares
def wait_for_select_options(driver, select_id, min_options=2, timeout=25):
    WebDriverWait(driver, timeout).until(
        lambda d: len(Select(d.find_element(By.ID, select_id)).options) >= min_options
    )

def wait_for_grade_carregar(driver, timeout=30):
    WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((By.ID, "gradeCurricular"))
    )
    WebDriverWait(driver, timeout).until(
        lambda d: len(d.find_elements(By.XPATH, "//div[@id='gradeCurricular']//tr")) > 0
    )

def esperar_overlay_sumir(driver, timeout=30):
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.blockUI.blockOverlay"))
        )
    except Exception:
        print("Overlay não desapareceu.")

def esperar_overlay_e_clique(driver, elemento, timeout=30):
    try:
        # Aguarda que o overlay desapareça
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.blockUI.blockOverlay"))
        )
        # Move para o elemento e clica
        driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
        WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(elemento)).click()
    except Exception as e:
        print(f"Erro ao tentar clicar no elemento: {e}")

def fechar_popup_erro_se_existir(driver):
    try:
        popup = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.XPATH, "//div[contains(@class, 'ui-dialog') and contains(., 'Dados não encontrados')]")
            )
        )
        botao_fechar = popup.find_element(By.XPATH, ".//span[text()='Fechar']/..")
        botao_fechar.click()
        WebDriverWait(driver, 5).until(EC.staleness_of(popup))
        return True
    except Exception:
        return False

def esperar_botao_buscar_habilitado(driver, timeout=15):
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']/.."))
    )

def extrair_disciplinas(soup):
    """
    Função para extrair disciplinas obrigatórias e optativas de uma página carregada.
    """
    grade_div = soup.find('div', {'id': 'gradeCurricular'})
    if not grade_div:
        return {"Obrigatórias": [], "Optativas": []}

    structured_data = {"Obrigatórias": [], "Optativas": []}

    tables = grade_div.find_all('table')
    for table in tables:
        table_title_row = table.find('tr', style=lambda x: x and 'rgb(16, 148, 171)' in x)
        if not table_title_row:
            continue
        table_title = table_title_row.get_text(strip=True)
        is_obrigatoria = "Obrigatórias" in table_title

        rows = table.find_all('tr', style=lambda x: x and 'height' in x)
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 8:
                discipline_data = {
                    "Código": cells[0].get_text(strip=True),
                    "Nome": cells[1].get_text(strip=True),
                    "Créditos Aula": cells[2].get_text(strip=True),
                    "Créditos Trabalho": cells[3].get_text(strip=True),
                    "CH": cells[4].get_text(strip=True),
                    "CE": cells[5].get_text(strip=True),
                    "CP": cells[6].get_text(strip=True),
                    "ATPA": cells[7].get_text(strip=True),
                }

                if is_obrigatoria:
                    structured_data["Obrigatórias"].append(discipline_data)
                else:
                    structured_data["Optativas"].append(discipline_data)

    return structured_data

def coletar_dados(num_unidades=3):
    driver = webdriver.Chrome()
    URL = "https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275"
    driver.get(URL)

    unidades = []

    try:
        wait_for_select_options(driver, "comboUnidade")
        select_unidades = Select(driver.find_element(By.ID, "comboUnidade"))

        for i in range(1, num_unidades + 1):
            select_unidades.select_by_index(i)
            unidade_nome = select_unidades.first_selected_option.text.strip()
            print(f"\nProcessando unidade: {unidade_nome}")
            unidade = Unidade(unidade_nome)

            wait_for_select_options(driver, "comboCurso")
            select_cursos = Select(driver.find_element(By.ID, "comboCurso"))
            cursos_options = [opt for opt in select_cursos.options if opt.get_attribute('value') != '']

            for option in cursos_options:
                try:
                    select_cursos.select_by_value(option.get_attribute('value'))
                    curso_nome = option.text.strip()

                    buscar_btn = esperar_botao_buscar_habilitado(driver)
                    buscar_btn.click()

                    WebDriverWait(driver, 30).until(
                        EC.element_to_be_clickable((By.ID, "step4-tab"))
                    )

                    if fechar_popup_erro_se_existir(driver):
                        continue

                    esperar_overlay_sumir(driver)
                    grade_tab = driver.find_element(By.ID, "step4-tab")
                    esperar_overlay_e_clique(driver, grade_tab)

                    wait_for_grade_carregar(driver)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    disciplinas = extrair_disciplinas(soup)

                    curso = Curso(curso_nome, unidade_nome, "N/A", "N/A", "N/A")
                    curso.disciplinas_obrigatorias.extend(disciplinas["Obrigatórias"])
                    curso.disciplinas_optativas.extend(disciplinas["Optativas"])

                    unidade.adicionar_curso(curso)

                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

                except Exception as e:
                    print(f"Erro no curso {option.text}: {e}")
                    aba_dados = driver.find_element(By.ID, "step1-tab")
                    esperar_overlay_e_clique(driver, aba_dados)

            unidades.append(unidade)

    finally:
        driver.quit()

    return unidades


In [57]:
class Curso:
    def __init__(self, nome, unidade, tipo, nivel, turno):
        self.nome = nome
        self.unidade = unidade
        self.tipo = tipo
        self.nivel = nivel
        self.turno = turno
        self.disciplinas_obrigatorias = []  # Lista de disciplinas obrigatórias
        self.disciplinas_optativas = []    # Lista de disciplinas optativas


In [58]:
# Classes
class Unidade:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []

    def adicionar_curso(self, curso):
        self.cursos.append(curso)

In [59]:
# Execução
if __name__ == "__main__":
    dados = coletar_dados(num_unidades=3)
    for unidade in dados:
        print(f"\nUnidade: {unidade.nome}")
        for curso in unidade.cursos:
            print(f"  Curso: {curso.nome}")
            print(f"    Disciplinas obrigatórias: {curso.disciplinas_obrigatorias}")



Processando unidade: Escola de Artes, Ciências e Humanidades - ( EACH )

Processando unidade: Escola de Comunicações e Artes - ( ECA )

Processando unidade: Escola de Educação Física e Esporte - ( EEFE )

Unidade: Escola de Artes, Ciências e Humanidades - ( EACH )
  Curso: Bacharelado em Biotecnologia (Ciclo Básico) - integral
    Disciplinas obrigatórias: [{'Código': 'ACH0021', 'Nome': 'Tratamento e Análise de Dados / Informações', 'Créditos Aula': '2', 'Créditos Trabalho': '0', 'CH': '30', 'CE': '', 'CP': '', 'ATPA': ''}, {'Código': 'ACH0041', 'Nome': 'Resolução de Problemas I', 'Créditos Aula': '4', 'Créditos Trabalho': '0', 'CH': '60', 'CE': '', 'CP': '', 'ATPA': ''}, {'Código': 'ACH5501', 'Nome': 'Introdução à Biotecnologia', 'Créditos Aula': '4', 'Créditos Trabalho': '4', 'CH': '180', 'CE': '', 'CP': '', 'ATPA': ''}, {'Código': 'ACH5511', 'Nome': 'Cálculo I', 'Créditos Aula': '4', 'Créditos Trabalho': '0', 'CH': '60', 'CE': '', 'CP': '', 'ATPA': ''}, {'Código': 'ACH5521', 'Nome'

In [34]:
class JupiterWebScraper:
    """
    Classe principal para orquestrar o scraping de dados e fornecer métodos de consulta.
    """
    def __init__(self):
        self.unidades = []
        self.all_courses = []
        self.all_disciplines = {} # Dicionário para armazenar disciplinas por código para busca rápida

    def carregar_dados(self, driver, limite_unidades=None):
        """
        Inicia o processo de carregamento de todas as unidades, cursos e disciplinas.
        """
        print("Iniciando carregamento de dados das unidades, cursos e disciplinas...")
        self.unidades = UnidadeUSP.carregar_unidades(driver, limite=limite_unidades)
        print(unidade)
        
        # Popula all_courses e all_disciplines para facilitar as consultas
        for unidade in self.unidades:
            for curso in unidade.cursos:
                self.all_courses.append(curso)
                for disciplina in curso.disciplinas:
                    # Adiciona a disciplina se não existir, ou atualiza se já existir
                    if disciplina.codigo not in self.all_disciplines:
                        self.all_disciplines[disciplina.codigo] = disciplina
                    # Garante que o nome do curso seja adicionado apenas uma vez por disciplina
                    if curso.nome not in self.all_disciplines[disciplina.codigo].cursos_associados:
                        self.all_disciplines[disciplina.codigo].cursos_associados.append(curso.nome)
        print("Dados carregados com sucesso!")

    # 1. Lista de cursos por unidades
    def listar_cursos_por_unidade(self):
        """Lista todos os cursos para cada unidade carregada."""
        print("\n--- Lista de Cursos por Unidade ---")
        if not self.unidades:
            print("Nenhuma unidade carregada. Por favor, carregue os dados primeiro.")
            return

        for unidade in self.unidades:
            print(f"\nUnidade: {unidade.nome}")
            if unidade.cursos:
                for curso in unidade.cursos:
                    print(unidade)
                    print(f"  - {curso.nome}")
            else:
                print("  Nenhum curso encontrado para esta unidade.")
 # 2. Dados de um determinado curso
    def dados_curso(self, nome_curso):
        """Exibe os dados detalhados de um curso específico, incluindo suas disciplinas."""
        print(f"\n--- Dados do Curso: '{nome_curso}' ---")
        found = False
        for curso in self.all_courses:
            if curso.nome.lower() == nome_curso.lower():
                print(f"Nome do Curso: {curso.nome}")
                if curso.disciplinas:
                    print("Disciplinas:")
                    for disciplina in curso.disciplinas:
                        print(f"  - {disciplina}")
                else:
                    print("  Nenhuma disciplina encontrada para este curso.")
                found = True
                break
        if not found:
            print(f"Curso '{nome_curso}' não encontrado.")

    # 3. Dados de todos os cursos
    def dados_todos_cursos(self):
        """Exibe os dados de todos os cursos carregados, com suas respectivas disciplinas."""
        print("\n--- Dados de Todos os Cursos ---")
        if not self.all_courses:
            print("Nenhum curso carregado. Por favor, carregue os dados primeiro.")
            return

        for curso in self.all_courses:
            print(f"\nCurso: {curso.nome}")
            if curso.disciplinas:
                print("  Disciplinas:")
                for disciplina in curso.disciplinas:
                    print(f"    - {disciplina.nome} (Créditos: {disciplina.creditos_total})")
            else:
                print("  Nenhuma disciplina encontrada para este curso.")

    # 4. Dados de uma disciplina, inclusive quais cursos ela faz parte
    def dados_disciplina(self, codigo_ou_nome_disciplina):
        """Exibe os dados detalhados de uma disciplina específica e os cursos que a utilizam."""
        print(f"\n--- Dados da Disciplina: '{codigo_ou_nome_disciplina}' ---")
        found_discipline = None
        for disc_code, disciplina in self.all_disciplines.items():
            if disc_code.lower() == codigo_ou_nome_disciplina.lower() or \
               disciplina.nome.lower() == codigo_ou_nome_disciplina.lower():
                found_discipline = disciplina
                break
        
        if found_discipline:
            print(f"Nome: {found_discipline.nome}")
            print(f"Código: {found_discipline.codigo}")
            print(f"Créditos de Aula: {found_discipline.creditos_aula}")
            print(f"Créditos de Trabalho: {found_discipline.creditos_trabalho}")
            print(f"Créditos Totais: {found_discipline.creditos_total}")
            print("Presente nos Cursos:")
            if found_discipline.cursos_associados:
                for curso_nome in sorted(found_discipline.cursos_associados): # Ordena para melhor leitura
                    print(f"  - {curso_nome}")
            else:
                print("  Nenhum curso associado encontrado.")
        else:
            print(f"Disciplina '{codigo_ou_nome_disciplina}' não encontrada.")

    # 5. Disciplinas que são usadas em mais de um curso
    def disciplinas_multiuso(self):
        """Lista as disciplinas que são utilizadas em mais de um curso."""
        print("\n--- Disciplinas Usadas em Mais de Um Curso ---")
        multi_course_disciplines = {}
        for disc_code, disciplina in self.all_disciplines.items():
            # Usa set para contar cursos únicos (evita duplicatas se uma disciplina for carregada múltiplas vezes)
            if len(set(disciplina.cursos_associados)) > 1: 
                multi_course_disciplines[disc_code] = disciplina
        
        if multi_course_disciplines:
            for disc_code, disciplina in multi_course_disciplines.items():
                print(f"\nDisciplina: {disciplina.nome} (Código: {disciplina.codigo})")
                print("  Presente nos Cursos:")
                for curso_nome in sorted(set(disciplina.cursos_associados)): # Ordena para melhor leitura
                    print(f"    - {curso_nome}")
        else:
            print("Nenhuma disciplina encontrada que seja usada em mais de um curso.")

    # 6. Outras consultas relevantes
    def cursos_com_mais_disciplinas(self, top_n=5):
        """Lista os top 'N' cursos com o maior número de disciplinas."""
        print(f"\n--- Top {top_n} Cursos com Mais Disciplinas ---")
        if not self.all_courses:
            print("Nenhum curso carregado. Por favor, carregue os dados primeiro.")
            return

        sorted_courses = sorted(self.all_courses, key=lambda c: len(c.disciplinas), reverse=True)
        
        for i, curso in enumerate(sorted_courses[:top_n]):
            print(f"{i+1}. {curso.nome} - {len(curso.disciplinas)} disciplinas")

    def unidades_com_mais_cursos(self, top_n=5):
        """Lista as top 'N' unidades com o maior número de cursos."""
        print(f"\n--- Top {top_n} Unidades com Mais Cursos ---")
        if not self.unidades:
            print("Nenhuma unidade carregada. Por favor, carregue os dados primeiro.")
            return

        sorted_unidades = sorted(self.unidades, key=lambda u: len(u.cursos), reverse=True)
        
        for i, unidade in enumerate(sorted_unidades[:top_n]):
            print(f"{i+1}. {unidade.nome} - {len(unidade.cursos)} cursos")

    def disciplinas_com_mais_creditos(self, top_n=5):
        """Lista as top 'N' disciplinas com o maior número de créditos totais."""
        print(f"\n--- Top {top_n} Disciplinas com Mais Créditos Totais ---")
        if not self.all_disciplines:
            print("Nenhuma disciplina carregada. Por favor, carregue os dados primeiro.")
            return
        
        sorted_disciplines = sorted(self.all_disciplines.values(), key=lambda d: d.creditos_total, reverse=True)
        
        for i, disciplina in enumerate(sorted_disciplines[:top_n]):
            print(f"{i+1}. {disciplina.nome} (Código: {disciplina.codigo}) - {disciplina.creditos_total} créditos")


In [35]:
# from selenium import webdriver

def main():
    print("=== Bem-vindo ao Sistema de Consulta JupiterWeb ===")
    
    # Iniciar o webdriver
    print("Iniciando o WebDriver...")
    driver = webdriver.Chrome()  # Certifique-se de ter o WebDriver configurado corretamente
    driver.get("https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275")

    try:
        # Criar instância do scraper
        scraper = JupiterWebScraper()
        
        # Carregar dados iniciais
        limite_unidades = input("Digite o número máximo de unidades a carregar (ou pressione Enter para carregar todas): ")
        limite_unidades = int(limite_unidades) if limite_unidades.strip().isdigit() else None
        scraper.carregar_dados(driver, limite_unidades)
        
        # Menu interativo
        while True:
            print("\n--- Menu Principal ---")
            print("1. Listar cursos por unidade")
            print("2. Dados de um curso específico")
            print("3. Dados de todos os cursos")
            print("4. Dados de uma disciplina específica")
            print("5. Disciplinas usadas em mais de um curso")
            print("6. Top cursos com mais disciplinas")
            print("7. Top unidades com mais cursos")
            print("8. Top disciplinas com mais créditos")
            print("9. Sair")
            
            opcao = input("Escolha uma opção: ")
            
            if opcao == '1':
                scraper.listar_cursos_por_unidade()
            elif opcao == '2':
                nome_curso = input("Digite o nome do curso: ")
                scraper.dados_curso(nome_curso)
            elif opcao == '3':
                scraper.dados_todos_cursos()
            elif opcao == '4':
                nome_ou_codigo_disciplina = input("Digite o nome ou código da disciplina: ")
                scraper.dados_disciplina(nome_ou_codigo_disciplina)
            elif opcao == '5':
                scraper.disciplinas_multiuso()
            elif opcao == '6':
                top_n = input("Quantos cursos listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.cursos_com_mais_disciplinas(top_n)
            elif opcao == '7':
                top_n = input("Quantas unidades listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.unidades_com_mais_cursos(top_n)
            elif opcao == '8':
                top_n = input("Quantas disciplinas listar? (padrão: 5): ")
                top_n = int(top_n) if top_n.strip().isdigit() else 5
                scraper.disciplinas_com_mais_creditos(top_n)
            elif opcao == '9':
                print("Encerrando o programa. Até logo!")
                break
            else:
                print("Opção inválida. Tente novamente.")
    
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    finally:
        print("Encerrando o WebDriver...")
        driver.quit()

if __name__ == "__main__":
    main()


=== Bem-vindo ao Sistema de Consulta JupiterWeb ===
Iniciando o WebDriver...


Digite o número máximo de unidades a carregar (ou pressione Enter para carregar todas):  3


Iniciando carregamento de dados das unidades, cursos e disciplinas...
Ocorreu um erro: cannot access local variable 'unidade' where it is not associated with a value
Encerrando o WebDriver...
